In [95]:
from main import Matrix, RowVector, ColumnVector

接下来是思考题，高斯消元法。

首先我们需要读取矩阵。

文件里面的方程大概长这样：

```
x+10y-12z=0
12x+18z=121
-x+18.3y-21.4z=-99.5
```

我们需要把它转化成矩阵的形式：

```
1 10 -12 0
12 0 18 121
-1 18.3 -21.4 -99.5
```

并且这只是一个示例，实际上变量不一定是x,y,z，方程的个数也不一定是3个。变量可能是`var_1`或者`VarOne`，只要是 Python 里面能合法成为变量名的都可以。

那么我们怎么提取这些系数呢？

我们可以观察到，每个方程等号右侧都是数字，左侧每个加号或者减号的右侧都是数字——如果不是的话就补1。此外，开头如果不是减号，就补加号。


In [96]:
from typing import Literal


with open('equations.txt', 'r', encoding='utf-8') as f:
    equations = []
    vals = []
    for i in f:
        if not i.startswith('-'):
            i=f"+{i}"
        var = ''
        eq = {}
        for _ in range(len(i)):
            j=i[_]
            
            if j in ('+', '-'):
                # add this into equations
                if var:
                    eq[var] = eval(opr + num) if num else eval(f'{opr}1')  # type: ignore
                num = ''
                var = ''
                in_var = False
                opr: Literal['+', '-'] = j
            
            elif j == '=':
                vals.append(eval(i[_+1:]))
                eq[var] = eval(opr + num) if num else eval(f'{opr}1')  # type: ignore
                print(eq)
                equations.append(eq.copy())
                break
            elif in_var:  # type: ignore
                var += j
            elif j in ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '.'):
                num += j  # type: ignore
            else:
                in_var = True
                var += j

{'x': 1, 'y': 10, 'z': -12}
{'x': 12, 'z': 18}
{'x': -1, 'y': 18.3, 'z': -21.4}


In [97]:
# test
print(equations)
print(vals)

[{'x': 1, 'y': 10, 'z': -12}, {'x': 12, 'z': 18}, {'x': -1, 'y': 18.3, 'z': -21.4}]
[0, 121, -99.5]


如此，我们已经能把方程读取出来了，现在转化为矩阵。

注意到，equations里面的每个元素都是一个字典，字典的键是变量名，值是系数。我们求每个字典的键的并集，就是所有的变量名。然后我们把每个方程的系数按照变量名的顺序排列，就得到了矩阵。

然后给这个矩阵右侧加上一列，就是等号右侧的数字。这就是所求的增广矩阵。

In [98]:
keys_set = set()
for i in equations:
    keys_set.update(i.keys())
keys = sorted(keys_set)
m=Matrix(*(RowVector(*(eq.get(k, 0) for k in keys)) for eq in equations))
m.insert_column(column=ColumnVector(*vals), index=len(keys))
print(m)

1	10	-12	0
12	0	18	121
-1	18.3	-21.4	-99.5


矩阵已经有了，接下来我们需要进行高斯消元。

In [99]:
def GaussJordanElimination(m: Matrix):
    for i in range(m.count_rows()):
        # make the diagonal element 1
        pivot = m(i, i)
        for j in range(m.count_columns()):
            m[j+i*m.count_columns()] /= pivot
        for j in range(m.count_rows()):
            if i == j:
                continue
            # make the elements in column i 0 except the diagonal element
            factor = m(row=j, column=i)
            for k in range(m.count_columns()):
                m[k+j*m.count_columns()] -= factor * m[k+i*m.count_columns()]
    return m
res = GaussJordanElimination(m.copy())
print(res)


1.0	0.0	0.0	32.236559139784916
0.0	1.0	0.0	-20.94623655913976
0.0	0.0	1.0	-14.768817204301055


这样我们就完成了高斯消元法的实现。

下面我们还需要将矩阵转化为方程组的形式，输出$\LaTeX$代码。

In [100]:
from typing import Sequence


def out_latex(m: Matrix, var_names: list[str]):
    if m.count_columns() != len(var_names) + 1:
        raise ValueError('The number of columns in the matrix should be equal to the number of variables + 1')
    print('\\begin{align}')
    for i in range(m.count_rows()):
        out = ''
        for j in range(m.count_columns()):
            num = m(column=j, row=i)
            if j == m.count_columns() - 1:
                out += f'&= {str(m(column=m.count_columns() - 1, row=i))} \\\\'
                print(out)
                break
            if num == 0:
                continue
            elif isinstance(num, complex):
                out += f"+({num}) {var_names[j]}"
            elif num > 0 and out:
                out += '+'
            else:
                out += '-'
            out += var_names[j] if abs(num) == 1 else str(abs(num)) + var_names[j]
            out += " "
    print('\\end{align}')

out_latex(m, keys)
print('\\\\\n\\text{Solutions: }\\\\')
out_latex(res, keys)

\begin{align}
-x +10y -12z &= 0 \\
-12x +18z &= 121 \\
-x +18.3y -21.4z &= -99.5 \\
\end{align}
\\
\text{Solutions: }\\
\begin{align}
-x &= 32.236559139784916 \\
-y &= -20.94623655913976 \\
-z &= -14.768817204301055 \\
\end{align}


解析成：

$$
\begin{align}
x +10y -12z &= 0 \\
12x +18z &= 121 \\
-x +18.3y -21.4z &= -99.5 \\
\end{align}
\\
\text{Solutions: }\\
\begin{align}
x &= 32.236559139784916 \\
y &= -20.94623655913976 \\
z &= -14.768817204301055 \\
\end{align}
$$

这样就完成了。